In [1]:
pip install split-folders

Note: you may need to restart the kernel to use updated packages.


In [2]:
import zipfile
import splitfolders
import os
import tensorflow as tf

2023-12-05 23:44:58.220535: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-05 23:44:58.716125: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 23:44:58.716207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 23:44:58.792383: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 23:44:58.936472: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# Extract the archive
local_zip = 'dataset_capstone.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [4]:
base_dir = 'data/dataset_capstone/'
splitfolders.ratio(base_dir, output = 'data/dataset_capstone/dataset', seed = 1337, ratio=(0.8, 0.2), group_prefix=None)
train_dir = 'data/dataset_capstone/dataset/train'
val_dir = 'data/dataset_capstone/dataset/val'

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
training_datagen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(
                    rescale=1./255)

In [6]:
train_generator = training_datagen.flow_from_directory(
	train_dir,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=60
)

validation_generator = validation_datagen.flow_from_directory(
	val_dir,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=60
)

Found 7678 images belonging to 5 classes.
Found 1922 images belonging to 5 classes.


In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

2023-12-05 23:45:26.452910: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-05 23:45:26.545789: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-05 23:45:26.545865: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-05 23:45:26.551670: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-05 23:45:26.551722: I external/local_xla/xla/stream_executor

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

In [9]:
model.compile(loss = 'categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(), metrics=['accuracy'])

In [10]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [11]:
# Train the model
history = model.fit(train_generator, epochs=500, steps_per_epoch=120, validation_data = validation_generator, verbose = 1, validation_steps=10)

Epoch 1/500


2023-12-05 23:45:44.229262: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2023-12-05 23:45:45.140991: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.91GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-05 23:45:45.309306: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.91GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-05 23:45:47.524924: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f6f75171ec0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-05 23:45:47.525077: I external/local_xla/xla/service/service.

  1/120 [..............................] - ETA: 20:51 - loss: 1.6129 - accuracy: 0.1833

2023-12-05 23:45:52.241699: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.31GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


  7/120 [>.............................] - ETA: 57s - loss: 1.8730 - accuracy: 0.2119

2023-12-05 23:45:56.116837: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.84GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-05 23:45:56.255374: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.84GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


  8/120 [=>............................] - ETA: 2:37 - loss: 1.8417 - accuracy: 0.2050

2023-12-05 23:46:02.102395: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.23GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-05 23:46:02.270108: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.23GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


120/120 [==============================] - 95s 709ms/step - loss: 1.6259 - accuracy: 0.1946 - val_loss: 1.6079 - val_accuracy: 0.2050
Epoch 2/500
120/120 [==============================] - 79s 656ms/step - loss: 1.6103 - accuracy: 0.1964 - val_loss: 1.6093 - val_accuracy: 0.2033
Epoch 3/500
120/120 [==============================] - 75s 620ms/step - loss: 1.6097 - accuracy: 0.2074 - val_loss: 1.6116 - val_accuracy: 0.2067
Epoch 4/500
120/120 [==============================] - 72s 599ms/step - loss: 1.6072 - accuracy: 0.2238 - val_loss: 1.6078 - val_accuracy: 0.2250
Epoch 5/500
120/120 [==============================] - 73s 609ms/step - loss: 1.6050 - accuracy: 0.2309 - val_loss: 1.5999 - val_accuracy: 0.2700
Epoch 6/500
120/120 [==============================] - 73s 608ms/step - loss: 1.6046 - accuracy: 0.2334 - val_loss: 1.5998 - val_accuracy: 0.2417
Epoch 7/500
120/120 [==============================] - 72s 599ms/step - loss: 1.5995 - accuracy: 0.2359 - val_loss: 1.5942 - val_accurac

In [15]:
pip install tensorflowjs

INFO: pip is looking at multiple versions of tensorstore to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.4/244.4 kB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 MB 1.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 2.0 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 1.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 2.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.0/558.0 kB 2.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.1 MB/s

In [17]:
import tensorflowjs

In [18]:
import time
saved_model_path = "./{}.h5".format(int(time.time()))

model.save(saved_model_path)

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./

2023-12-06 06:23:08.651383: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 06:23:08.652210: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 06:23:08.786064: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
saved_model_path = "./{}.h5".format(int(time.time()))

history.save(saved_model_path)

AttributeError: 'History' object has no attribute 'save'

In [ ]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./